<a href="https://colab.research.google.com/github/aakshataaa/Machine_failure/blob/main/Machine_Failure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# train_model_simple.py
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import os

print("Loading dataset...")
df = pd.read_csv('train.csv')

print("Basic preprocessing...")
# Convert target variable
df['Failure'] = df['Failure'].map({'Yes': 1, 'No': 0})

# Simple feature engineering
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y %H:%M')
df['Hour'] = df['Date'].dt.hour
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Encode operator
le = LabelEncoder()
df['Operator_encoded'] = le.fit_transform(df['Operator'])

# Select simple features
features = [
    'Temperature', 'Humidity', 'Operator_encoded', 'Hour', 'DayOfWeek',
    'Measure1', 'Measure2', 'Measure3', 'Hours Since Previous Failure'
]

X = df[features]
y = df['Failure']

print(f"Training samples: {len(X)}")
print(f"Failure cases: {y.sum()}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train simple model
print("Training model...")
model = RandomForestClassifier(
    n_estimators=50,
    max_depth=8,
    random_state=42,
    class_weight='balanced'
)

model.fit(X_train, y_train)

# Evaluate
accuracy = model.score(X_test, y_test)
print(f"Model accuracy: {accuracy:.4f}")

# Save model
model_data = {
    'model': model,
    'label_encoder': le,
    'features': features,
    'accuracy': accuracy
}

with open('model_simple.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("Model saved as 'model_simple.pkl'")
print("Training completed!")

Loading dataset...
Basic preprocessing...
Training samples: 7905
Failure cases: 75
Training model...
Model accuracy: 0.9975
Model saved as 'model_simple.pkl'
Training completed!


In [ ]:
# app_simple.py
from flask import Flask, render_template, request, jsonify
import pandas as pd
import pickle
from datetime import datetime

app = Flask(__name__)

# Load model
try:
    with open('model_simple.pkl', 'rb') as f:
        model_data = pickle.load(f)
    model = model_data['model']
    le = model_data['label_encoder']
    features = model_data['features']
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if model is None:
        return jsonify({'error': 'Model not loaded'})

    try:
        data = request.get_json()

        # Create input DataFrame
        input_df = pd.DataFrame([{
            'Temperature': float(data['Temperature']),
            'Humidity': float(data['Humidity']),
            'Operator_encoded': le.transform([data['Operator']])[0],
            'Hour': pd.to_datetime(data['Date']).hour,
            'DayOfWeek': pd.to_datetime(data['Date']).dayofweek,
            'Measure1': int(data['Measure1']),
            'Measure2': int(data['Measure2']),
            'Measure3': int(data['Measure3']),
            'Hours Since Previous Failure': float(data['Hours Since Previous Failure'])
        }])

        # Make prediction
        prediction = model.predict(input_df)[0]
        probability = model.predict_proba(input_df)[0][1]

        status = "Failure Likely" if prediction == 1 else "Normal Operation"

        return jsonify({
            'prediction': status,
            'probability': round(probability * 100, 2),
            'risk_level': 'High' if probability > 0.7 else 'Medium' if probability > 0.4 else 'Low'
        })

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '_main_':
    # To run this in Colab, you'll need a tunneling service like ngrok
    # Ensure ngrok is running and authenticated - you might need to add your authtoken
    # Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
    # Run the following command in a cell to add it:
    # !ngrok config add-authtoken YOUR_AUTHTOKEN_HERE
    app.run(debug=True, port=5000)

Model loaded successfully!


In [ ]:
!pip install dash pandas plotly scikit-learn pyngrok # Ensure necessary libraries are installed including pyngrok

import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
import base64
import io
from pyngrok import ngrok # Import ngrok
import threading
import time
from datetime import datetime


# Load the trained model and data if available
try:
    with open("model_simple.pkl", "rb") as f:
        model_data = pickle.load(f)
    model = model_data["model"]
    le = model_data["label_encoder"]
    features = model_data["features"]
    print("Model and Label Encoder loaded successfully!")

    # Load the training data to get operator options (optional, if needed for other purposes)
    # Using a simplified list of operators for the dashboard as requested
    # Include all operators from 1 to 7 as requested
    operators = [f'Operator{i}' for i in range(1, 8)]
    # If you have actual data loaded, you could combine and sort unique operators from there too:
    # df_train = pd.read_csv('train.csv') # Uncomment if needed
    # df_train['Date'] = pd.to_datetime(df_train['Date'], format='%d-%m-%Y %H:%M', errors='coerce') # Uncomment if needed
    # operators = sorted(list(set(df_train['Operator'].unique().tolist() + [f'Operator{i}' for i in range(1, 8)]))) # Example combining

    # Refit the LabelEncoder with the operators used in the dashboard dropdown
    # Make sure the encoder can handle all operators from 1 to 7
    le.fit(operators)
    print("LabelEncoder refitted with dashboard operators.")


except Exception as e:
    print(f"Error loading model or data or refitting LabelEncoder: {e}")
    model = None
    le = None
    features = None
    # Default to all operators 1-7 if data loading fails
    operators = [f"Operator{i}" for i in range(1, 8)]


# 1. --- Initialize the Dash App ---
# ------------------------------------
# Using a modern, clean Bootstrap theme for styling.
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.title = "Machine Failure Prediction Dashboard"


# No custom dark theme CSS needed for white background

# 2. --- Define Data Structures (Sample Data for Dashboard Visualizations) ---
# ---------------------------------------------------------------------------

# Data for Historical Performance (replace with actual data if available)
historical_data = pd.DataFrame({
    "date": pd.to_datetime(["2024-01", "2024-02", "2024-03", "2024-04", "2024-05", "2024-06", "2024-07", "2024-08"]),
    "failures": [12, 8, 15, 6, 11, 9, 13, 7],
    "predictions": [14, 9, 16, 7, 12, 10, 14, 8],
    "accuracy": [85.7, 88.9, 93.8, 85.7, 91.7, 90.0, 92.9, 87.5],
})

# Data for Feature Importance (replace with actual model output if available)
feature_importance_data = pd.DataFrame({
    "feature": ["Temperature", "Humidity", "Measure 1", "Measure 2", "Measure 3", "Operator"],
    "importance": [0.32, 0.24, 0.18, 0.15, 0.08, 0.03],
})

# Data for Risk Distribution (replace with actual prediction results if available)
risk_distribution_data = pd.DataFrame({
    "name": ["Low Risk", "Medium Risk", "High Risk"],
    "value": [65, 25, 10],
    "color": ["#2ecc71", "#f39c12", "#e74c3c"],
})

# Data for Failure Rate by Day of Week (replace with actual data)
failure_dayofweek_data = pd.DataFrame({
    "day": ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
    "failure_rate": [5, 7, 6, 8, 9, 4, 3] # Sample percentages
})

# Data for Failure Rate by Hour of Day (replace with actual data)
failure_hour_data = pd.DataFrame({
    "hour": list(range(24)),
    "failure_rate": [1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 4, 3, 2, 2, 1, 1, 1] # Sample percentages
})


# 3. --- Define Reusable Components & Chart Figures ---
# -----------------------------------------------------

# Function to create styled "Stat Cards"
def create_stat_card(title, value, trend, color):
    return html.Div(
        className="four columns",
        style={"padding": "10px", "margin": "0 5px"},
        children=[
            html.Div(
                style={
                    "backgroundColor": "white",
                    "padding": "20px",
                    "borderRadius": "8px",
                    "boxShadow": "0 2px 10px rgba(0,0,0,0.1)",
                    "borderLeft": f"4px solid {color}",
                },
                children=[
                    html.P(title, style={"fontSize": "16px", "color": "#6c757d"}),
                    html.H4(value, style={"margin": "5px 0", "fontSize": "28px"}),
                    html.P(trend, style={"fontSize": "12px", "color": "#adb5bd"}),
                ],
            )
        ],
    )

# --- Create Chart Figures ---

# Historical Performance Line Chart
fig_historical = px.line(
    historical_data,
    x="date",
    y=["failures", "predictions"],
    labels={"value": "Count", "date": "Month"},
    title="Historical Performance Trends",
)
fig_historical.update_layout(
    plot_bgcolor="white", paper_bgcolor="white", font_color="#333"
)

# Feature Importance Bar Chart
fig_importance = px.bar(
    feature_importance_data,
    x="feature",
    y="importance",
    color="feature",
    labels={"importance": "Importance Score", "feature": "Feature"},
    title="Feature Importance Analysis",
)
fig_importance.update_layout(showlegend=False, plot_bgcolor="white", paper_bgcolor="white", font_color="#333")

# Risk Distribution Donut Chart
fig_risk = go.Figure(
    data=[
        go.Pie(
            labels=risk_distribution_data["name"],
            values=risk_distribution_data["value"],
            hole=0.6,
            marker_colors=risk_distribution_data["color"],
        )
    ]
)
fig_risk.update_layout(
    title="Fleet Risk Distribution",
    showlegend=True,
    annotations=[dict(text="Fleet", x=0.5, y=0.5, font_size=20, showarrow=False)],
    plot_bgcolor="white", paper_bgcolor="white", font_color="#333"
)

# Failure Rate by Day of Week Bar Chart
fig_dayofweek = px.bar(
    failure_dayofweek_data,
    x="day",
    y="failure_rate",
    labels={"day": "Day of Week", "failure_rate": "Failure Rate (%)"},
    title="Failure Rate by Day of Week",
    color_discrete_sequence=['purple']
)
fig_dayofweek.update_layout(yaxis_range=[0, 10], plot_bgcolor="white", paper_bgcolor="white", font_color="#333")

# Failure Rate by Hour of Day Bar Chart
fig_hour = px.bar(
    failure_hour_data,
    x="hour",
    y="failure_rate",
    labels={"hour": "Hour of Day", "failure_rate": "Failure Rate (%)"},
    title="Failure Rate by Hour of Day",
    color_discrete_sequence=['red']
)
fig_hour.update_layout(xaxis=dict(tickmode='linear'), yaxis_range=[0, 12], plot_bgcolor="white", paper_bgcolor="white", font_color="#333")


# 4. --- Define App Layout ---
# ----------------------------
app.layout = html.Div(
    style={"backgroundColor": "#f8f9fa", "padding": "20px"}, # Light background for the whole page
    children=[
        # Header
        html.Div(
            [
                html.H1(
                    "Machine Failure Prediction Dashboard",
                    style={"textAlign": "center", "color": "#343a40"},
                ),
                html.P(
                    "Real-time monitoring and predictive analytics for industrial equipment",
                    style={"textAlign": "center", "color": "#6c757d"},
                ),
            ],
            style={"marginBottom": "20px"},
        ),
        # Stat Cards Row
        html.Div(
            className="row",
            style={"marginBottom": "20px"},
            children=[
                create_stat_card("Model Accuracy", "89.3%", "+2.1% from last month", "#2ecc71"), # Replace with actual model accuracy
                create_stat_card("Active Machines", "247", "3 offline", "#3498db"),
                create_stat_card("Predicted Failures", "8", "Next 30 days", "#e74c3c"), # Replace with actual prediction count
                create_stat_card("Uptime", "97.2%", "Last 30 days", "#9b59b6"),
            ],
        ),
        # Main Content Grid (Prediction Controls and Charts)
        html.Div(
            className="row",
            children=[
                # Left Column: Prediction Controls
                html.Div(
                    className="four columns",
                    style={"backgroundColor": "white", "padding": "20px", "borderRadius": "8px", "marginBottom": "20px"}, # Light background
                    children=[
                        html.H4("Input Features for Prediction"),
                        html.P("Adjust the values below to get a live failure prediction."),

                        # Temperature Input
                        html.Label("Temperature (°C)"),
                        dcc.Input(id="temp-input", type="number", value=70, step=0.1, style={'width': '100%', 'marginBottom': '10px'}),

                        # Humidity Input
                        html.Label("Humidity (%)"),
                        dcc.Input(id="humidity-input", type="number", value=50, step=0.1, style={'width': '100%', 'marginBottom': '10px'}),

                        # Operator Dropdown
                        html.Label("Operator"),
                        dcc.Dropdown(
                            id="operator-dropdown",
                            options=[{"label": op, "value": op} for op in operators],
                            value=operators[0] if operators else None,
                            style={'marginBottom': '10px'}
                        ),

                        # Measure1 Input
                        html.Label("Measure 1"),
                        dcc.Input(id="measure1-input", type="number", value=1000, style={'width': '100%', 'marginBottom': '10px'}),

                        # Measure2 Input
                        html.Label("Measure 2"),
                        dcc.Input(id="measure2-input", type="number", value=1, style={'width': '100%', 'marginBottom': '10px'}),

                        # Measure3 Input
                        html.Label("Measure 3"),
                        dcc.Input(id="measure3-input", type="number", value=1, style={'width': '100%', 'marginBottom': '10px'}),

                         # Hours Since Previous Failure Input
                        html.Label("Hours Since Previous Failure"),
                        dcc.Input(id="hours-since-failure-input", type="number", value=100, step=0.1, style={'width': '100%', 'marginBottom': '20px'}),

                        html.Button('Predict', id='predict-button', n_clicks=0, style={'width': '100%', 'padding': '10px', 'backgroundColor': '#28a745', 'color': 'white', 'border': 'none', 'borderRadius': '4px', 'cursor': 'pointer'}),


                    ],
                ),
                # Right Column: Live Prediction Output
                html.Div(
                    className="eight columns",
                    style={"backgroundColor": "white", "padding": "20px", "borderRadius": "8px", "marginBottom": "20px"}, # Light background
                    children=[
                        html.H4("Live Prediction Result"),
                        html.Div(id="live-prediction-output", style={'marginTop': '20px'}),
                    ],
                ),
            ],
        ),
        # Full-width Charts Section
        html.Div(
            className="row",
            style={"marginTop": "20px"},
            children=[
                html.Div(className="six columns", children=[dcc.Graph(figure=fig_historical)]),
                html.Div(className="six columns", children=[dcc.Graph(figure=fig_importance)]),
            ],
        ),
         html.Div(
            className="row",
            style={"marginTop": "20px"},
            children=[
                html.Div(className="six columns", children=[dcc.Graph(figure=fig_dayofweek)]),
                html.Div(className="six columns", children=[dcc.Graph(figure=fig_hour)]),
            ],
        ),
        html.Div(
            className="row",
            style={"marginTop": "20px"},
            children=[
                html.Div(className="twelve columns", children=[dcc.Graph(figure=fig_risk)])
            ]
        )
    ],
)


# 5. --- Define Callbacks for Interactivity ---
# ---------------------------------------------

# Callback to update the live prediction output
@app.callback(
    Output("live-prediction-output", "children"),
    [Input("predict-button", "n_clicks")],
    [
        dash.dependencies.State("temp-input", "value"),
        dash.dependencies.State("humidity-input", "value"),
        dash.dependencies.State("operator-dropdown", "value"),
        dash.dependencies.State("measure1-input", "value"),
        dash.dependencies.State("measure2-input", "value"),
        dash.dependencies.State("measure3-input", "value"),
        dash.dependencies.State("hours-since-failure-input", "value"),
    ],
)
def update_live_prediction(n_clicks, temperature, humidity, operator, measure1, measure2, measure3, hours_since_failure):
    if n_clicks == 0 or model is None or le is None or features is None:
        return html.Div("Click 'Predict' to get a failure prediction.")

    try:
        # Prepare input data for the model
        input_data = {
            'Temperature': float(temperature),
            'Humidity': float(humidity),
            'Operator': operator, # Use original operator for encoding
            'Measure1': float(measure1),
            'Measure2': float(measure2),
            'Measure3': float(measure3),
            'Hours Since Previous Failure': float(hours_since_failure),
            # Add placeholder values for Hour and DayOfWeek to match training features
            'Hour': 0, # Placeholder value
            'DayOfWeek': 0 # Placeholder value
        }

        # Encode operator
        # Check if the operator is in the fitted classes, if not, refit or handle
        # We are refitting with the 'operators' list defined above, which now includes 1-7
        le.fit(operators) # Refit le with the potentially updated operators list

        operator_encoded = le.transform([input_data['Operator']])[0]
        input_data['Operator_encoded'] = operator_encoded
        del input_data['Operator'] # Remove original operator


        # Create DataFrame, ensuring correct feature order
        input_df = pd.DataFrame([input_data])[features]

        # Make prediction
        prediction = model.predict(input_df)[0]
        probability = model.predict_proba(input_df)[0][1]

        status = "Failure Likely" if prediction == 1 else "Normal Operation"
        risk_level = 'High' if probability > 0.7 else 'Medium' if probability > 0.4 else 'Low'
        color = "#e74c3c" if risk_level == 'High' else "#f39c12" if risk_level == 'Medium' else "#2ecc71"


        # Display prediction result
        return html.Div(
            style={"padding": "20px", "border": f"2px solid {color}", "borderRadius": "8px", "backgroundColor": f"{color}20"},
            children=[
                html.H5("Prediction Result:", style={"color": color}),
                html.H3(status, style={"color": color, "textAlign": "center"}),
                html.Hr(),
                html.P(f"Failure Probability: {probability:.1%}", style={"fontWeight": "bold"}),
                html.P(f"Risk Level: {risk_level}", style={"fontWeight": "bold", "color": color}),
                # Add recommendations based on risk_level or specific inputs if desired
            ],
        )

    except Exception as e:
        return html.Div(f"Error during prediction: {e}", style={'color': 'red'})


# 6. --- Run the Server ---
# -------------------------
if __name__ == "_main_":
    # To run this in Colab, you'll need a tunneling service like ngrok
    # Ensure ngrok is running and authenticated - you might need to add your authtoken
    ngrok.set_auth_token("32pY8j7eJTpj7gkPpouj0vxrJLS_6iPYhYi5pTnLkJaYhefPn") # Add the authtoken here
    try:
        print("Attempting to start ngrok tunnel...")
        # Kill any process running on port 8050
        import os
        os.system('fuser -k 8050/tcp')
        import time
        time.sleep(5) # Give the system a moment to free the port

        public_url = ngrok.connect(8050).public_url
        print(f" * Dash app running on: {public_url}")

        # Start the Flask app in a separate thread
        # Use a separate thread as app.run() is blocking
        threading.Thread(target=app.run, kwargs={'port': 8050, 'debug': True, 'use_reloader': False}).start()

    except Exception as e:
        print(f"Error starting ngrok tunnel or Dash app: {e}")
        print("Please ensure you have pyngrok installed and your ngrok authtoken configured.")

Model and Label Encoder loaded successfully!
LabelEncoder refitted with dashboard operators.
